In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transform as transform
from torchvision.models import vgg19

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.downsample = nn.Sequential(
            
        )


In [ ]:
class Residual(nn.Module):
    def __init__(self):
        super().__init__()